# Filtering the characteristics' file and associating it with the segmentation masks

This notebook can be viewed online :
* Live, with MyBinder [at this URL](https://mybinder.org/v2/git/https%3A%2F%2Fgit.sophia.mines-paristech.fr%2Foie%2Fbdappv.git/HEAD?labpath=metadata.ipynb)
* Static with NbViewer [at this URL](https://nbviewer.org/urls/git.sophia.mines-paristech.fr/oie/bdappv/-/raw/master/metadata.ipynb)

In this notebook, we associate the raw characteristics file with the segmentation masks. We apply three filters : 
- Internal consistency : verify that the characteristics reported are coherent, according to a certain amount of criteria
- Ambiguity : If the installation is associated with a mask that contains more than one polygon
- External consistency : if the relationship between the surface estimated from the polygon and the surface reported in the characteristics file do not match

The variable `controlled` indicates whether the installation passes the internal consistency criteria. The variables `IGNControlled` and `GoogleControlled` indicate whether the ambiguity and external consistency criteria are met, with respect to the masks of the IGN and Google campaign respectively. 

The application of these filters can be reproduced using this notebook. These filters have generated the `metadata.csv` file. You can also apply your own filters and conduct your own analyses using this notebook. 

<b> Prerequisites </b> 

- Specify the path to the BDAPPV database. It is then assumed that the structure is left unchanged.

In [ ]:
# To download the data folder from the Zenodo repository 
# Skip this cell if you downloaded and placed it in the folder.
!wget 'https://zenodo.org/record/7358126/files/data.zip?download=1' -O 'data.zip'
# unzip the file
!unzip 'data.zip' 
# delete the zip file
!rm 'data.zip'

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import plotly.graph_objects as go
import os
from PIL import Image
from lib.utils import *
#from rasterio import features, Affine
#import shapely
#from shapely import geometry
import tqdm

In [ ]:
JS_FILE_TEMPLATE="data/replication/campaign-{campaign}/polygon-analysis.json"

In [ ]:
table = pd.read_csv(os.path.join(os.path.join("data","raw"), "raw-metadata.csv")) # it is assumed that the notebook is executed from the repository.
table.head()

In [ ]:
table.shape

## Internal consistency

We consider the database and filter the seemingly erroneous installations. To do so we plot a few descriptive statistics. Tag the variables that are coherent with an attribute `controlled`. Then, additional attributes will be made for the observations conssitent with IGN and consistent with Google (both handled independently).

In [ ]:
table.describe()

Surface, arrays and installed capacity edits. 

Single variable edits :
- Tag observations that contain 0 array
- Tag observations for which the surface is lower than 0 or greater than 100 000
- Tag observations for which the instaled capacity is lower than 0

Bivariate edits :
- Tag observations for which the capacity per array is lower than a minimum capacity (default : set to 0)
- Tag observations for which the ratio between the surface and the installed capacity is strange and adjust the ratio if necessary

In [ ]:
def single_variable_filter(df, key, minimum = -1e6, maximum = 1e6):
    """
    returns an additional column indicating whether the observation 
    lies within the defined boundaries
    """
    # create a new key
    coherent_key = 'coherent_{}'.format(key)
    
    # condition
    df[coherent_key] = (df[key] < maximum) & (df[key] > minimum)
    
    print('Number of installations filtered for the {}:'.format(key), df[df[coherent_key] == False].shape[0])
    
    return None

# tilt : below 90 degrees
single_variable_filter(table, 'tilt', maximum = 60.)

# installed capacity : positive
single_variable_filter(table, 'kWp', minimum = 0)

# surface
single_variable_filter(table, 'surface', minimum = 0, maximum = 1e5)

# number of arrays : ositive
single_variable_filter(table, 'countArrays', minimum = 0)


In [ ]:
# filter the items for which the capacity per array is lower than min capacity. 
# set to 0 by default
min_capacity = 0 
table['coherent_array'] = table['kWp'] / table['countArrays'] > min_capacity
table[table['coherent_array'] == False].shape[0]

In [ ]:
# consider the filtered table 
table['coherent'] = table['coherent_tilt'] * table['coherent_kWp'] * table['coherent_surface'] * table['coherent_countArrays'] * table['coherent_array']
table[table['coherent'] == False].shape[0]

In [ ]:
px.scatter(table[table["coherent"] == True], 'surface', 'kWp', hover_data = ['idInstallation'])

In [ ]:
px.histogram((table[table['coherent'] == True]['kWp'] / 1000) / table[table['coherent'] == True]['surface'], log_y = True)

In [ ]:
# rescale the observations for which the ratio is too high (above 500)

threshold = 0.3425

table['rescaled'] = (table['kWp'] / 1000) / table['surface'] >= threshold
table['rescaled_kWp'] = table['rescaled'] * 1000 * table['kWp'] + (1 - table["rescaled"]) * table['kWp'] 
table[table['rescaled'] == True].shape[0]

In [ ]:
table[['kWp', 'rescaled_kWp', 'rescaled']]

In [ ]:
table

In [ ]:
px.scatter(table[table['coherent'] == True], 'surface', 'rescaled_kWp', hover_data = ['idInstallation'])

In [ ]:
filter_back_installations, filter_back_indices = [36075, 10851, 4959, 31875], []

for index in filter_back_installations:
    filter_back_indices.append(table[table['idInstallation'] == index].index.item())
    
for index in filter_back_indices:
    kWp = table.loc[index, 'kWp']
    table.loc[index, 'rescaled_kWp'] = kWp

In [ ]:
px.scatter(table[table['coherent'] == True], 'surface', 'rescaled_kWp', hover_data = ['idInstallation'])

In [ ]:
surface_max = 50.
kWp_min = 378000

rescale_back = table[(table['coherent'] == True) & (table['surface'] <= surface_max) & (table['rescaled_kWp'] >= kWp_min)].index
print(rescale_back)
for index in rescale_back:
    kWp = table.loc[index, 'kWp']
    table.loc[index, 'rescaled_kWp'] = kWp

In [ ]:
px.scatter(table[table['coherent'] == True], 'surface', 'rescaled_kWp', hover_data = ['idInstallation'])

In [ ]:
surface_min = 500.
kWp_max = 10000

rescale_back = table[(table['coherent'] == True) & (table['surface'] >= surface_min) & (table['rescaled_kWp'] <= kWp_max)].index
print(rescale_back)

for index in rescale_back:
    kWp = table.loc[index, 'kWp']
    table.loc[index, 'rescaled_kWp'] = kWp * 1000

In [ ]:
table['true_kWp'] = table['rescaled_kWp'] / 1000
px.scatter(table[table['coherent'] == True], 'surface', 'true_kWp', hover_data = ['idInstallation'])

In [ ]:
# remaining ids are marked as strange

surface_max = 6000.
kWp_min = 1000

table['coherent_surface_kWp'] = True 
rescale_back = table[(table['coherent'] == True) & (table['surface'] <= surface_max) & (table['true_kWp'] >= kWp_min)].index
print(rescale_back)

for index in rescale_back:
    table.loc[index, 'coherent_surface_kWp'] = False

In [ ]:
table['controlled'] = table['coherent'] * table['coherent_surface_kWp']

px.scatter(table[table['controlled'] == True], 'surface', 'true_kWp', hover_data = ['idInstallation'])

In [ ]:
# we finally clean the table of all the intermediary attributes.
table = table.drop(['coherent_tilt', 'coherent_kWp', 'coherent_surface',
       'coherent_countArrays', 'coherent_array', 'coherent', 'rescaled',
       'rescaled_kWp', 'true_kWp', 'coherent_surface_kWp'], axis = 1)

In [ ]:
table.shape[0] - table[table['controlled'] == True].shape[0]

In [ ]:
table.head()

## Association with the masks

We create two new variables `hasUniqueGoogleMask` and `hasUniqueIGNMask` to associate the installations with the segmentation masks. We indicate whether the masks only contains one polygon or not.

In [ ]:
def has_unique_poly(campaign, ids) :
    # Load results for the given campaign
    js = load_js(JS_FILE_TEMPLATE.format(campaign=campaign))

    # Build a dict
    res_polys={r.id: r for r in js}
    
    return list(id in res_polys and len(res_polys[id].polygons) == 1 for id in ids)

In [ ]:
table["hasUniqueGoogleMask"] = has_unique_poly("google", table.identifiant)

In [ ]:
table["hasUniqueIGNMask"] = has_unique_poly("ign", table.identifiant)

In [ ]:
table[table['hasUniqueGoogleMask'] == True].shape[0], table[table['hasUniqueIGNMask'] == True].shape[0] 

## External consistency 

Finally, we check the adequacy between the masks and the characteristics by plotting the true projected surface against the estimated projected surface. For the values that do not match, we label the installations


In [ ]:
table.head()

In [ ]:
def filter_external_consistency(campaign, table, gsd):
    """
    returns the table with an attribute filtering 
    """
    
    # Load AREAs
    js = load_js(JS_FILE_TEMPLATE.format(campaign=campaign.lower()))
    
    # Build a dict of areas
    areas={r.id : sum(p.area for p in r.polygons) for r in js}
    
    # computes the projection from the mask and from the database
    projection_items = []
    targets = table[table['hasUnique{}Mask'.format(campaign)] == True].index

    for index in tqdm.tqdm(targets):

        id = table.loc[index, 'identifiant']
        est_proj = areas[id] * (gsd ** 2)    
        table_proj = table.loc[index, 'surface'] * np.cos(table.loc[index,'tilt'] * np.pi / 180)

        projection_items.append([table.loc[index, 'identifiant'], est_proj, table_proj])


    projection = pd.DataFrame(projection_items, columns = ['identifiant', 'estimated', 'target'])
    projection['ratio'] = projection['estimated'] / projection['target']
    
    correct_identifiants = projection[(projection['ratio'] < 1.25) & (projection['ratio'] > 0.75)]['identifiant'].values

    table['{}Controlled'.format(campaign)] = False
    
    for identifiant in correct_identifiants:
        
        # index 
        index = table[table['identifiant'] == identifiant].index        
        # change the value depending on whether the
        # observation is also controlled
        
        table.loc[index, '{}Controlled'.format(campaign)] = table.loc[index, 'controlled'] * True
        
    # return the projection table to generate the plots
        
    return projection

In [ ]:
projection_ign = filter_external_consistency('IGN', table, 0.2)
projection_google = filter_external_consistency('Google', table, 0.1)

In [ ]:
table[table['IGNControlled'] == True].shape[0], table[table['GoogleControlled'] == True].shape[0]

In [ ]:
table.head()

# Conclusion

Throughout this quality control, we added indicator variables at three levels : 
- `controlled` : to indicate whether a characteristic is coherent,
- `{provider}Controlled` to indicate whether the characteristics is unique and abides by external consistency

Using these filters, we have 3127 installations attached and coherent with a IGN mask and 7753 installations coherent with a Google mask. Setting aside the masks, we have filtered 645 incoherent installations.

The table below displays the remaining number of installations after each filter is applied. The final column indicates the total count of filtered characteristics that are unambiguously with a mask.

|| Google | IGN | <i> Removed Google (%)  / Removed IGN (%) </i> |
|---|---|---|---|
Raw | 28408 | 28408 | <i> 0 (0) / 0 (0) </i> |
Internal consistency | 27780 | 27780 | <i> 628 (2.21%) / 628 (2.21%) </i> |
Mask uniqueness | 10523 | 5883 | <i> 17257 (62.12%) / 21897 (78.82%) </i> |
External consistency | 8019 | 3658 | <i> 2504 (23.80%) / 2225 (37.82%) </i> |


In [ ]:
table = table.drop(["hasUniqueGoogleMask", 'hasUniqueIGNMask'], axis = 1)
table.columns

In [ ]:
# Export the file
# table.to_csv(os.path.join(os.path.join("data","validation"), "metadata.csv"), index = None)

## Generation of the plots and statistics

In [ ]:
# plot as an illustration the estimated installed capacity against the true installed capacity

correct_ign = projection_ign[(projection_ign['ratio'] < 1.25) & (projection_ign['ratio'] > 0.75)]['identifiant'].values
projection_ign['correct'] = projection_ign['identifiant'].isin(correct_ign)

correct_identifiants = projection_google[(projection_google['ratio'] < 1.25) & (projection_google['ratio'] > 0.75)]['identifiant'].values
projection_google['correct'] = projection_google['identifiant'].isin(correct_identifiants)

# for index in range(projection_google.shape[0]):
#    
#    # get the estimation and the true value
#    estimation, target = projection_google.loc[index, 'estimated'], projection_google.loc[index, 'target']
#    distance = np.linalg.norm(estimation - target) #compute the element wise distance
#    projection_google.loc[index, 'distance'] = distance

for i in range(projection_google.shape[0]):
    projection_google.loc[i, 'label'] = "Filtered" if projection_google.loc[i,'correct'] == True else "Not filtered"

for i in range(projection_ign.shape[0]):
    projection_ign.loc[i, 'label'] = "Filtered" if projection_ign.loc[i,'correct'] == True else "Not filtered"


In [ ]:
# Correlation coefficients

targets = projection_google["target"].values
preds = projection_google['estimated'].values


filtered_targets = projection_google[projection_google["correct"] == True]["target"].values
filtered = projection_google[projection_google["correct"] == True]["estimated"].values
targets.shape, preds.shape

np.corrcoef(targets, preds), np.corrcoef(filtered_targets, filtered)

In [ ]:
# Correlation coefficients (IGN)

targets = projection_ign["target"].values
preds = projection_ign['estimated'].values


filtered_targets = projection_ign[projection_ign["correct"] == True]["target"].values
filtered = projection_ign[projection_ign["correct"] == True]["estimated"].values
targets.shape, preds.shape

np.corrcoef(targets, preds), np.corrcoef(filtered_targets, filtered)

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")


with open("data/validation/campaign-google/click-analysis-thres=1.0.json", "r") as read_file: # considérer le jeu de données complet
    dataPoint = json.load(read_file)
    
clicks = list(sum([x['clicks'] for x in dataPoint], [])) # some images have more than one click
ListPerfClick = np.array([x['score'] for x in clicks])

with open("data/validation/campaign-google/polygon-analysis-thres=1.0.json", "r") as read_file:
    dataPoly= json.load(read_file)
    
# extract the number of actors rom the raw data
with open("data/raw/input-google.json", "r") as read_file:
    dataRaw= json.load(read_file)

# create a dictionnary with the info on the polygons

stats = {
    item['id'] : dict( 
        id=item['id'],
        nb_poly = len(item['polygons']),
        nb_clicks = len(item['clicks']),
        nb_poly_actors = len(set(poly['action']['actorId'] for poly in item['polygons'])))
    for item in dataRaw
}
    
# extract the scores and convert into relative scores by dividing by the number of actors.
    
items = [(x['polygons'], stats[x['id']]['nb_poly_actors']) for x in dataPoly] # extracts the polygons and the number of actors
scores  = [[z['score'] / item[1] for z in item[0]] for item in items] # compute the relative PAC for each polygon
# convert as a unidimensional np.array
ListPerfPoly = np.array(list(sum(scores, [])))

#%%
fig,ax=plt.subplots(1,3,figsize=[23,6])

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)

t=ax[0].hist(ListPerfClick,np.arange(0,15,1))
values=t[0]
base=t[1]
plt.ylabel("Count")
ax_bisPoint = ax[0].twinx()
values = np.append(values,0)
ax_bisPoint.plot( base, 100*np.cumsum(values)/ np.cumsum(values)[-1], color='darkorange', marker='o', linestyle='-', markersize = 1, label = "Cumulative Histogram" )
ax_bisPoint.set_ylabel("proportion [%]")
ax[0].set_ylabel("Count [-]")
ax[0].set_xlabel("Pixel annotation consensus (PAC) [-]")
ax_bisPoint.set_ylim([0,100])
ax[0].set_title('Distribution of the PAC for image classification \n (phase 1)')
ax_bisPoint.legend()

t=ax[1].hist(ListPerfPoly)
values=t[0]
base=t[1]
ax_bisPoly = ax[1].twinx()
values = np.append(values,0)
ax_bisPoly.plot( base, 100*np.cumsum(values)/ np.cumsum(values)[-1], color='darkorange', marker='o', linestyle='-', markersize = 1, label = "Cumulative Histogram" )
ax_bisPoly.set_ylabel("proportion [%]")
ax[1].set_ylabel("Count [-]")
ax[1].set_xlabel("Relative pixel annotation consensus (PAC) [-]")
ax_bisPoint.set_ylim([0,100])
ax[1].set_title('Distribution of the relative PAC for polygon annotations \n (phase 2)')
ax_bisPoint.legend()

#ax[2].set_title('Analysis of the comparison between \n annotation and metadata')

ax[2].scatter(projection_google[projection_google["correct"] == False]['estimated'], projection_google[projection_google["correct"] == False]['target'], color = "blue", alpha = 0.1, label = "Discarded installations")
ax[2].scatter(projection_ign[projection_ign["correct"] == False]['estimated'], projection_ign[projection_ign["correct"] == False]['target'], color = "blue", alpha = 0.1)

ax[2].plot(projection_google['target'], projection_google['target'], label = "Perfect association", color = "black", alpha = 0.5)

ax[2].scatter(projection_google[projection_google["correct"] == True]['estimated'], projection_google[projection_google["correct"] == True]['target'], color = "red", label = "Filtered installations")
ax[2].scatter(projection_ign[projection_ign["correct"] == True]['estimated'], projection_ign[projection_ign["correct"] == True]['target'], color = "red")

ax[2].set_title("Correlation between the computed and recorded projected surface")
ax[2].set_xlabel("Projected surface reported in the registry [m²]")
ax[2].set_ylabel('Projected surface computed from the mask [m²]')
ax[2].legend()

plt.savefig('validation.pdf')

plt.show()